In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from copy import deepcopy
from torch.optim import AdamW
import hmac
import hashlib

# --- HufuNet Autoencoder ---

class HufuEncoder(nn.Module):
    """Encoder part of HufuNet autoencoder."""
    def __init__(self, in_channels=3, latent_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, latent_dim, 3, stride=2, padding=1),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.encoder(x)


class HufuDecoder(nn.Module):
    """Decoder part of HufuNet autoencoder."""
    def __init__(self, in_channels=3, latent_dim=64):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, in_channels, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(x)


class HufuAutoencoder(nn.Module):
    """Complete HufuNet autoencoder."""
    def __init__(self, in_channels=3, latent_dim=64):
        super().__init__()
        self.encoder = HufuEncoder(in_channels, latent_dim)
        self.decoder = HufuDecoder(in_channels, latent_dim)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


# --- Classe Principale HufuNet DDPM ---

class HufuDDPM:
    def __init__(self, model_id, device="cuda", secret_key="2020"):
        self.device = device
        self.model_id = model_id
        self.secret_key = secret_key

        # Chargement du modele
        self.pipeline = DDPMPipeline.from_pretrained(model_id)
        self.unet = self.pipeline.unet.to(device)
        self.scheduler = self.pipeline.scheduler

        # Configuration par defaut
        self.config = {
            "in_channels": 3,
            "latent_dim": 64,
            "lr": 1e-4,
            "lr_ae": 1e-3,         # Learning rate for autoencoder pretraining
            "ae_epochs": 1,        # Epochs to pretrain autoencoder
            "epochs": 100,           # Epochs for finetuning with watermark
            "mse_threshold": 0.01,  # Threshold for ownership verification
        }

        self.saved_keys = {}

    def _get_conv_params(self, model):
        """
        Extract all conv weight parameters from the model.
        Returns a flat parameter vector and layer info for reconstruction.
        """
        all_params = []
        layers_info = []

        for name, param in model.named_parameters():
            if 'conv' in name.lower() and 'weight' in name:
                all_params.append(param.view(-1))
                layers_info.append({
                    'name': name,
                    'shape': param.shape,
                    'numel': param.numel()
                })

        if len(all_params) == 0:
            # Fallback: use all parameters
            for name, param in model.named_parameters():
                if 'weight' in name:
                    all_params.append(param.view(-1))
                    layers_info.append({
                        'name': name,
                        'shape': param.shape,
                        'numel': param.numel()
                    })

        param_vector = torch.cat(all_params)
        return param_vector, layers_info

    def _hash_position(self, decoder_value, index, total_params):
        """
        Compute hash-based position for embedding using HMAC-SHA256.
        """
        message = str(int(decoder_value * 1000) ^ index).encode()
        mac = hmac.new(self.secret_key.encode(), message, hashlib.sha256)
        position = int(mac.hexdigest(), 16) % total_params
        return position

    def _train_autoencoder(self, autoencoder, dataloader, epochs=5, freeze_decoder=False):
        """
        Pre-train the HufuNet autoencoder.
        """
        print("--- Pre-training HufuNet Autoencoder ---")

         # Freeze or unfreeze decoder
        for param in autoencoder.decoder.parameters():
            param.requires_grad = not freeze_decoder

        if freeze_decoder:
            print("Decoder is frozen — training encoder only.")
            optimizer = torch.optim.Adam(autoencoder.encoder.parameters(), lr=self.config["lr_ae"])
        else:
            print("Decoder is trainable — training full autoencoder.")
            optimizer = torch.optim.Adam(autoencoder.parameters(), lr=self.config["lr_ae"])




        criterion = nn.MSELoss()

        for epoch in range(epochs):
            pbar = tqdm(dataloader, desc=f"AE Epoch {epoch+1}/{epochs}")
            total_loss = 0
            for images, _ in pbar:
                images = images.to(self.device)
                # Normalize to [0, 1] for autoencoder
                images_norm = (images + 1) / 2

                optimizer.zero_grad()
                _, reconstructed = autoencoder(images_norm)
                loss = criterion(reconstructed, images_norm)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                pbar.set_postfix(Loss=loss.item())

            print(f"Epoch {epoch+1} - Avg Loss: {total_loss/len(dataloader):.4f}")
        for param in autoencoder.decoder.parameters():
            param.requires_grad = True

        return autoencoder
    def get_encoder_parameters(self, encoder):
        """
        Extract encoder parameters into a flat vector for embedding.
        """

        encoder_params = []
        for name, param in encoder.named_parameters():
            if 'weight' in name:
                encoder_params.append(param.view(-1).detach())
        encoder_vector = torch.cat(encoder_params)
        return encoder_vector

    def get_decoder_parameters(self, decoder):
        """
        Extract decoder parameters into a flat vector for hashing.
        """
        decoder_params = []
        for name, param in decoder.named_parameters():
            if 'weight' in name:
                decoder_params.append(param.view(-1).detach())
        decoder_vector = torch.cat(decoder_params)
        return decoder_vector

    def embedded_positons_in_model(self, model, encoder_vector, decoder_vector):
        """
        Embeds encoder parameters into the model weights using hash-based positioning.
        """
        param_vector, layer_info = self._get_conv_params(model)
        total_params = param_vector.numel()
        watermark_size = encoder_vector.numel()

        bitmap = torch.zeros(total_params, dtype=torch.bool, device=self.device)
        embedded_positions = []

        with torch.no_grad():
            for i in tqdm(range(min(watermark_size, total_params // 10)), desc="Embedding"):
                decoder_val = decoder_vector[i % len(decoder_vector)].item()
                position = self._hash_position(decoder_val, i, total_params)

                original_pos = position
                while bitmap[position]:
                    position = (position + 1) % total_params
                    if position == original_pos:
                        break

                embedded_positions.append(position)
                bitmap[position] = True


        return embedded_positions, bitmap, layer_info

    def _embed_params_into_model(self, model, encoder_vector, embedded_positions):
        """
        Embeds encoder parameters into model weights at the specified positions.
        Returns the modified model.
        """
        # Build direct mapping: global_param_index -> encoder_value
        position_to_encoder_value = {
            pos: encoder_vector[i]
            for i, pos in enumerate(embedded_positions)
            if i < len(encoder_vector)
        }

        # Get model layers info
        _, layers_info = self._get_conv_params(model)

        # Embed into model
        param_idx = 0
        with torch.no_grad():
            for info in layers_info:
                for name, param in model.named_parameters():
                    if name == info['name']:
                        param_flat = param.view(-1)
                        for j in range(info['numel']):
                            global_idx = param_idx + j
                            if global_idx in position_to_encoder_value:
                                param_flat[j] = position_to_encoder_value[global_idx]
                        break
                param_idx += info['numel']

        return model
    def _extract_and_evaluate(self, model, embedded_positions, decoder, dataloader):
        """
        Extracts parameters from model at given positions,
        reconstructs the encoder and full autoencoder,
        evaluates MSE on dataloader.
        Returns reconstructed autoencoder and avg MSE.
        """
        # Get current model parameters
        param_vector, _ = self._get_conv_params(model)

        # Extract values at embedded positions
        extracted_params = torch.zeros(len(embedded_positions), device=self.device)
        with torch.no_grad():
            for i, pos in enumerate(embedded_positions):
                if pos < len(param_vector):
                    extracted_params[i] = param_vector[pos]

        # Reconstruct encoder by loading extracted values into architecture
        reconstructed_encoder = HufuEncoder(
            in_channels=self.config["in_channels"],
            latent_dim=self.config["latent_dim"]
        ).to(self.device)



        offset = 0
        with torch.no_grad():
            for name, param in reconstructed_encoder.named_parameters():
                if 'weight' in name:
                    numel = param.numel()
                    chunk = extracted_params[offset:offset + numel]
                    if len(chunk) == numel:
                        param.copy_(chunk.view(param.shape))
                    offset += numel

        # Rebuild full autoencoder with reconstructed encoder + owner decoder
        reconstructed_autoencoder = HufuAutoencoder(
            in_channels=self.config["in_channels"],
            latent_dim=self.config["latent_dim"]
        ).to(self.device)
        reconstructed_autoencoder.encoder = reconstructed_encoder
        reconstructed_autoencoder.decoder = decoder

        # Evaluate MSE on dataloader
        reconstructed_autoencoder.eval()
        criterion = nn.MSELoss()
        total_loss = 0
        n_batches = 0

        with torch.no_grad():
            for images, _ in dataloader:
                images = images.to(self.device)
                images_norm = (images + 1) / 2
                _, reconstructed = reconstructed_autoencoder(images_norm)
                loss = criterion(reconstructed, images_norm)
                total_loss += loss.item()
                n_batches += 1
                if n_batches >= 10:
                    break

        avg_mse = total_loss / n_batches
        print(f"Reconstructed autoencoder MSE: {avg_mse:.4f}")

        return reconstructed_autoencoder, avg_mse
    def embed(self, dataloader):
        """
        Embeds HufuNet encoder into DDPM model weights.
        """
        print(f"--- Demarrage Embedding HufuNet ---")
        autoencoder = HufuAutoencoder(
            in_channels=self.config["in_channels"],
            latent_dim=self.config["latent_dim"]
        ).to(self.device)


        # 1. Train autoencoder
        autoencoder = self._train_autoencoder(autoencoder, dataloader, self.config["ae_epochs"])
        encoder = autoencoder.encoder
        decoder = autoencoder.decoder


        encoder_vector = self.get_encoder_parameters(encoder)
        decoder_vector = self.get_decoder_parameters(decoder)
        watermark_size = encoder_vector.numel()

        # 4. Get model parameters
        watermarked_unet = self.unet
        embedded_positions, bitmap, layers_info=self.embedded_positons_in_model( watermarked_unet, encoder_vector, decoder_vector)


        # 8. Fine-tune to maintain model quality
        print("\n--- Fine-tuning watermarked model ---")
        watermarked_unet.train()
        optimizer = torch.optim.AdamW(watermarked_unet.parameters(), lr=self.config["lr"])
        mse_loss = nn.MSELoss()

        for epoch in range(self.config["epochs"]):
            encoder_vector = self.get_encoder_parameters(encoder)

            watermarked_unet=self._embed_params_into_model(watermarked_unet, encoder_vector, embedded_positions)
            pbar = tqdm(dataloader, desc=f"Finetune Epoch {epoch+1}")



            for clean_images, _ in pbar:
                clean_images = clean_images.to(self.device)
                bs = clean_images.shape[0]

                noise = torch.randn_like(clean_images).to(self.device)
                timesteps = torch.randint(0, self.scheduler.config.num_train_timesteps, (bs,), device=self.device).long()
                noisy_images = self.scheduler.add_noise(clean_images, noise, timesteps)

                optimizer.zero_grad()
                noise_pred = watermarked_unet(noisy_images, timesteps).sample
                loss = mse_loss(noise_pred, noise)
                loss.backward()
                optimizer.step()

                pbar.set_postfix(Loss=loss.item())
            reconstructed_autoencoder, avg_mse=self._extract_and_evaluate(watermarked_unet, embedded_positions, decoder, dataloader)
            print(f"Epoch {epoch+1} - Finetune Loss: {loss.item():.4f} | Extracted Autoencoder MSE: {avg_mse:.4f}")
            if avg_mse <  self.config["mse_threshold"]:
                print(f"MSE= {avg_mse} is acceptable, stopping fine-tuning to preserve watermark integrity.")
                break
            else:
                print(f"MSE= {avg_mse} is too high, continuing fine-tuning to improve autoencoder reconstruction.")
                autoencoder = self._train_autoencoder(reconstructed_autoencoder, dataloader, self.config["ae_epochs"], freeze_decoder=True)
                encoder = autoencoder.encoder

        # Save keys
        self.saved_keys = {
            "autoencoder": reconstructed_autoencoder,
            "encoder": reconstructed_autoencoder.encoder,
            "decoder": reconstructed_autoencoder.decoder,
            # "encoder_vector": encoder_vector,
            # "decoder_vector": decoder_vector,
            "embedded_positions": embedded_positions,
            "layers_info": layers_info,
            "watermarked_unet": watermarked_unet,
        }
        return watermarked_unet, reconstructed_autoencoder

    # def extract(self, suspect_unet=None):
    #     """
    #     Extracts the embedded encoder from a suspect model.
    #     """
    #     if suspect_unet is None:
    #         suspect_unet = self.saved_keys["watermarked_unet"]
    #
    #     decoder_vector = self.saved_keys["decoder_vector"]
    #     embedded_positions = self.saved_keys["embedded_positions"]
    #     encoder_vector = self.saved_keys["encoder_vector"]
    #
    #     # Get model parameters
    #     param_vector, _ = self._get_conv_params(suspect_unet)
    #
    #     # Extract encoder parameters
    #     extracted_encoder = torch.zeros_like(encoder_vector[:len(embedded_positions)])
    #
    #     with torch.no_grad():
    #         for i, pos in enumerate(embedded_positions):
    #             if pos < len(param_vector) and i < len(extracted_encoder):
    #                 extracted_encoder[i] = param_vector[pos]
    #
    #     # Compute correlation/similarity with original encoder
    #     original_encoder = encoder_vector[:len(embedded_positions)].to(self.device)
    #     extracted_encoder = extracted_encoder.to(self.device)
    #
    #     mse = F.mse_loss(extracted_encoder, original_encoder).item()
    #     correlation = F.cosine_similarity(
    #         extracted_encoder.unsqueeze(0),
    #         original_encoder.unsqueeze(0)
    #     ).item()
    #
    #     print(f"Extraction MSE: {mse:.4f}")
    #     print(f"Correlation with original: {correlation:.4f}")
    #
    #     return mse, correlation, extracted_encoder
    def extract(self, model=None, dataloader=None):
        """
        Extracts the embedded encoder from a suspect model and tests its functionality
        by reconstructing images using the owner's decoder.
        """
        if model is None:
            model = self.saved_keys["watermarked_unet"]

        decoder = self.saved_keys["decoder"]
        embedded_positions = self.saved_keys["embedded_positions"]



        reconstructed_autoencoder, avg_mse=self._extract_and_evaluate(model, embedded_positions, decoder, dataloader)
        extracted_encoder = reconstructed_autoencoder.encoder


        return avg_mse, extracted_encoder

    # def verify(self, suspect_unet=None, test_dataloader=None):
    #     """
    #     Verifies ownership by testing if extracted encoder + owner's decoder
    #     can reconstruct images.
    #     """
    #     if suspect_unet is None:
    #         suspect_unet = self.saved_keys["watermarked_unet"]
    #
    #     decoder = self.saved_keys["decoder"]
    #     autoencoder = self.saved_keys["autoencoder"]
    #
    #     # Get extraction results
    #     mse, correlation, extracted_encoder = self.extract(suspect_unet)
    #
    #     # Reconstruct using owner's autoencoder for comparison
    #     if test_dataloader is not None:
    #         autoencoder.eval()
    #         criterion = nn.MSELoss()
    #         total_loss = 0
    #         n_batches = 0
    #
    #         with torch.no_grad():
    #             for images, _ in test_dataloader:
    #                 images = images.to(self.device)
    #                 images_norm = (images + 1) / 2
    #                 _, reconstructed = autoencoder(images_norm)
    #                 loss = criterion(reconstructed, images_norm)
    #                 total_loss += loss.item()
    #                 n_batches += 1
    #                 if n_batches >= 10:  # Limit batches for speed
    #                     break
    #
    #         avg_loss = total_loss / n_batches
    #         print(f"Autoencoder reconstruction MSE: {avg_loss:.4f}")
    #
    #     # Verification based on correlation
    #     is_owner = correlation > 0.5
    #     print(f"\nOwnership verified: {is_owner}")
    #
    #     return is_owner, correlation


/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- EXEMPLE D'EXECUTION ---

# 1. Setup Data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 2. Embedding HufuNet
hufu_defense = HufuDDPM("google/ddpm-cifar10-32")
watermarked_model = hufu_defense.embed(dataloader)

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00, 49.32it/s]


--- Demarrage Embedding HufuNet ---
--- Pre-training HufuNet Autoencoder ---
Decoder is trainable — training full autoencoder.


AE Epoch 1/1: 100%|██████████| 782/782 [00:03<00:00, 238.08it/s, Loss=0.00619]


Epoch 1 - Avg Loss: 0.0134


Embedding: 100%|██████████| 56160/56160 [00:01<00:00, 51882.58it/s]


...

Reconstructed autoencoder MSE: 0.2045
Epoch 98 - Finetune Loss: 0.0269 | Extracted Autoencoder MSE: 0.2045
MSE= 0.20451595336198808 is too high, continuing fine-tuning to improve autoencoder reconstruction.
--- Pre-training HufuNet Autoencoder ---
Decoder is frozen — training encoder only.


AE Epoch 1/1: 100%|██████████| 782/782 [00:03<00:00, 234.61it/s, Loss=0.00427]


Epoch 1 - Avg Loss: 0.0062


Finetune Epoch 99: 100%|██████████| 782/782 [01:01<00:00, 12.77it/s, Loss=0.0147]


Reconstructed autoencoder MSE: 0.0071
Epoch 99 - Finetune Loss: 0.0147 | Extracted Autoencoder MSE: 0.0071
MSE= 0.007083063432946801 is acceptable, stopping fine-tuning to preserve watermark integrity.


In [ ]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack_hufu(hufu_obj, dataloader, test_dataloader, epochs=5, lr=1e-4):
    """
    Tente de transferer la fonctionnalite du modele HufuNet vers un modele vierge.
    Verifie si le watermark survit.
    """
    device = hufu_obj.device

    # 1. Teacher (Gele)
    teacher_unet = hufu_obj.saved_keys["watermarked_unet"]
    teacher_unet.eval()
    for p in teacher_unet.parameters():
        p.requires_grad = False

    # 2. Student (Vierge - Meme architecture)
    print("\n--- Initialisation du Student ---")
    student_pipeline = DDPMPipeline.from_pretrained("google/ddpm-cifar10-32")
    student_unet = student_pipeline.unet.to(device)
    student_unet.train()



    mse_teacher, _ = hufu_obj.extract(teacher_unet, dataloader)
    print("\n[Check] Teacher: mse_hufu_teacher: {:.4f}".format(mse_teacher))

    mse_student, _ = hufu_obj.extract(student_unet, dataloader)
    print("\n[Check] Student (before): mse_hufu_student: {:.4f}".format(mse_student))

    optimizer = AdamW(student_unet.parameters(), lr=lr)
    noise_scheduler = hufu_obj.scheduler
    history = {"loss": [], "MSE": []}

    print(f"\n--- Distillation HufuNet ({epochs} epochs) ---")

    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0

        for clean_images, _ in pbar:
            clean_images = clean_images.to(device)
            bs = clean_images.shape[0]

            noise = torch.randn_like(clean_images).to(device)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bs,), device=device).long()
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with torch.no_grad():
                target_pred = teacher_unet(noisy_images, timesteps).sample

            student_pred = student_unet(noisy_images, timesteps).sample

            loss = F.mse_loss(student_pred, target_pred)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(Loss=loss.item())

        # Check correlation
        mse , _ = hufu_obj.extract(student_unet, dataloader)
        history["MSE"].append(mse)
        history["loss"].append(running_loss / len(dataloader))

        print(f"Fin Epoch {epoch+1} | Loss: {history['loss'][-1]:.4f} | mse_student: {mse:.4f}")



    return student_unet, history


In [4]:
# 5. Attaque par Distillation
student_res, stats = run_distillation_attack_hufu(hufu_defense, dataloader, test_dataloader, epochs=100)


--- Initialisation du Student ---


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00, 19.39it/s]


Reconstructed autoencoder MSE: 0.0440

[Check] Teacher: mse_hufu_teacher: 0.0440
Reconstructed autoencoder MSE: 0.0660

[Check] Student (before): mse_hufu_student: 0.0660

--- Distillation HufuNet (100 epochs) ---


Epoch 1: 100%|██████████| 782/782 [01:17<00:00, 10.15it/s, Loss=0.000948]


Reconstructed autoencoder MSE: 0.0696
Fin Epoch 1 | Loss: 0.0007 | mse_student: 0.0696


...

Reconstructed autoencoder MSE: 0.0759
Fin Epoch 35 | Loss: 0.0002 | mse_student: 0.0759


Epoch 36: 100%|██████████| 782/782 [01:16<00:00, 10.19it/s, Loss=0.000272]


Reconstructed autoencoder MSE: 0.0800
Fin Epoch 36 | Loss: 0.0002 | mse_student: 0.0800


Epoch 37:   3%|▎         | 21/782 [00:02<01:18,  9.75it/s, Loss=0.000168]


KeyboardInterrupt: 

In [ ]:
# 6. Visualisation des resultats
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(stats["loss"])
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.set_title("Distillation Loss")

ax2.plot(stats["correlation"])
ax2.axhline(y=0.5, color='r', linestyle='--', label='Threshold (0.5)')
ax2.axhline(y=0.0, color='g', linestyle='--', label='No correlation (0.0)')
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Correlation")
ax2.set_title("Encoder Correlation During Distillation")
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 7. Visualize Autoencoder Reconstruction
import matplotlib.pyplot as plt

autoencoder = hufu_defense.saved_keys["autoencoder"]
autoencoder.eval()

# Get some test images
test_images, _ = next(iter(test_dataloader))
test_images = test_images[:8].to(hufu_defense.device)
test_images_norm = (test_images + 1) / 2

with torch.no_grad():
    _, reconstructed = autoencoder(test_images_norm)

# Plot
fig, axes = plt.subplots(2, 8, figsize=(16, 4))

for i in range(8):
    # Original
    img = test_images_norm[i].cpu().permute(1, 2, 0).numpy()
    axes[0, i].imshow(img)
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Original')

    # Reconstructed
    rec = reconstructed[i].cpu().permute(1, 2, 0).numpy()
    axes[1, i].imshow(rec)
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Reconstructed')

plt.tight_layout()
plt.show()